In [1]:
import cenpy
import pandas as pd
import matplotlib.pyplot as plt
import mapclassify.classifiers as classifiers

In [2]:
schemes = {}
for classifier in classifiers.CLASSIFIERS:
    schemes[classifier.lower()] = getattr(classifiers, classifier)

In [ ]:
gdfs = []

for year in range(2013,2020):
    chi = cenpy.products.ACS(year).from_place('Chicago, IL', variables= ['B01003_001', 'B19013_001E', 'B25077_001E',
                                                'B03002_012E','B02008_001E', 'B02011_001E', 'B02009_001E',
                                                 'B17001_002E'], level='tract')
    chi['year'] = year
    gdfs.append(chi)

chi_gdf = pd.concat(gdfs)
acs_variables = {
'B01003_001E' : 'total_pop',
'B19013_001E' : 'median_household_income',
'B25077_001E' : 'median_home_value',
'B03002_012E' : 'n_hispanic',
'B02008_001E' : 'n_white',
'B02009_001E' : 'n_black',
'B02011_001E' : 'n_asian',
'B17001_002E' : 'n_poverty'}
chi_gdf.rename(columns=acs_variables, inplace=True)
chi_gdf.drop(['state', 'county'], axis=1, inplace=True)

/home/andrew/anaconda3/envs/eats/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Matched: Chicago, IL to Chicago city within layer Incorporated Places


/home/andrew/anaconda3/envs/eats/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_527413/734256529.py:4: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  chi = cenpy.products.ACS(year).from_place('Chicago, IL', variables= ['B01003_001', 'B19013_001E', 'B25077_001E',
/home/andrew/anaconda3/envs/eats/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/st

Matched: Chicago, IL to Chicago city within layer Incorporated Places


/home/andrew/anaconda3/envs/eats/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_527413/734256529.py:4: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  chi = cenpy.products.ACS(year).from_place('Chicago, IL', variables= ['B01003_001', 'B19013_001E', 'B25077_001E',
/home/andrew/anaconda3/envs/eats/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/st

Matched: Chicago, IL to Chicago city within layer Incorporated Places


/home/andrew/anaconda3/envs/eats/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_527413/734256529.py:4: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  chi = cenpy.products.ACS(year).from_place('Chicago, IL', variables= ['B01003_001', 'B19013_001E', 'B25077_001E',


In [ ]:
chi_gdf['p_poverty'] = chi_gdf.n_poverty/chi_gdf.total_pop
chi_gdf['p_black'] = chi_gdf.n_black/chi_gdf.total_pop
chi_gdf['p_white'] = chi_gdf.n_white/chi_gdf.total_pop
chi_gdf['p_hispanic'] = chi_gdf.n_hispanic/chi_gdf.total_pop
chi_gdf['p_asian'] = chi_gdf.n_asian/chi_gdf.total_pop

In [ ]:
def plot_time(df, column, cmap, scheme, k):
    classifier = schemes[scheme](df[column].dropna().values, k=k)
    f, axs = plt.subplots(ncols=len(df.year.unique()), figsize=(25,25))
    for i, year in enumerate(df.year.unique()):
        df[df.year == year].plot(column, cmap=cmap, scheme='user_defined',
                                 #legend_kwds={'loc': 'lower center'},
                                 classification_kwds={"bins": classifier.bins},
                                 ax=axs[i])
        axs[i].axis('off')
        axs[i].set_title(year)

    lines, labels = axs[-1].get_legend_handles_labels()
    f.legend(lines, labels, loc = 'lower center')
    return axs

In [ ]:
plot_time(chi_gdf,'p_poverty', 'Greens', 'quantiles', k=5)

In [ ]:
chi_gdf[chi_gdf.year == 2019].plot('p_black', figsize = (8,8), legend=True)

In [ ]:
chi_gdf.to_file("acs.geojson", driver='GeoJSON')